In [ ]:
!pip install OSMPythonTools

In [ ]:
from OSMPythonTools.overpass import Overpass

In [ ]:
import pandas as pd

In [ ]:
overpass = Overpass()

In [ ]:
result.elements()[0].tag("name")

In [ ]:
result = overpass.query(
    """area["ISO3166-1"="RU"]->.a1;
        (
          nwr[place="city"](area.a1);
          nwr[place="town"](area.a1);
        );
        out body;
    """,
    timeout=9000
)

In [ ]:
def get_region_name(lat, lon):
    if lat is None:
        return None
    if lon is None:
        return None
    result = overpass.query(
        f"""is_in({lat}, {lon});
           area._[admin_level="4"]; 
           out body;
        """,
        timeout=9000,
    )
    return result.elements()[0].tags()["name:ru"]

In [ ]:
data = [
    {
        "name": c.tag("name"),
        "start_date": c.tag("start_date"),
        "population": c.tag("population"),
        "admin_level": c.tag("admin_level"),
        "capital": c.tag("capital"),
        "lat": c.lat(),
        "lon": c.lon(),
        "region": get_region_name(c.lat(), c.lon())
    }
    for c in list(result.elements())
]

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df = df[df['region'].notna()]

In [ ]:
df

In [ ]:
df.to_csv("osm_cities_and_towns.csv", index=False)

# Caffee

In [ ]:
from tqdm.autonotebook import tqdm

In [ ]:
df = pd.read_csv('./Тестовый датасет/covid_data_test.csv', sep=',')

In [ ]:
cities = df["name"]
subjects = df["subject"]

In [ ]:
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

In [ ]:
amenities = [
    "bar",
#     "biergarten",
    "cafe",
    "fast_food",
#     "food_court",
#     "ice_cream",
    "pub",
    "restaurant",
    "clinic",
    "hospital",
    "pharmacy",
    "cinema",
    "nightclub",
#     "stripclub",
    "theatre",
#     "prison",
#     "fitness_centre",
#     "sports_centre",
#     "monastery",
    "place_of_worship"
]

In [ ]:
overpass = Overpass()

data = []

for (
    c,
    s,
) in tqdm(list(zip(cities, subjects))):
    stat = {"name": c, "subject": s}
    
    s = s.replace("Еврейская АО", "Еврейская Автономная область")
    s = s.replace("АО", "автономный округ")

    nominatim = Nominatim()
    areaId = nominatim.query(f"{c}, {s}, Россия").areaId()
    if areaId is None:
        areaId = nominatim.query(f"{c}, {s}, Украина").areaId()
    if areaId is None:
        continue
        print("Error", c, s)

    for a in amenities:
        query = overpassQueryBuilder(
            area=areaId, elementType="node", selector=f'"amenity"="{a}"', out="count"
        )
        result = overpass.query(query, timeout=9000)
        stat[a] = result.countElements()
        
    query = overpassQueryBuilder(
        area=areaId, elementType="node", selector='"railway"="halt"', out="count"
    )
    result = overpass.query(query, timeout=9000)
    stat["railway_halt"] = result.countElements()

    data.append(stat)

In [ ]:
osm_stat = pd.DataFrame(data)

In [ ]:
osm_stat.to_csv("osm_stat_train.csv", index=False)

In [ ]:
osm_stat

In [ ]:
for d in osm_stat.columns:
    print(d)
    print(osm_stat[osm_stat[d] == 0].shape)